In [1]:
import sys
import os

# Link BigPy
sys.path.insert(0, '/home/av/AI-Lab/BigPy/')
from bigpy.bench_report import *  # nopep8
from bigpy.inmem_md import InMemMD  # nopep8
import bigpy.pretty_print as pp  # nopep8

In [8]:
benches = frame_google_benchmarks('numpy_vs_cupy.json')
benches = benches[['name', 'mean_duration', 'size', 'backend']]
workloads = benches['name'].unique().tolist()
sizes = sorted(benches['size'].unique())

In [9]:
workloads

['generate_random_matrix',
 'moving_average',
 'pearson_correlations',
 'markov_clustering',
 'matrix_multiply',
 'singular_decomposition']

In [4]:
import plotly.express as px
import pandas as pd

In [11]:
for workload in workloads:
    cpus = filtered(benches, name=workload, backend='numpy')['mean_duration'].to_numpy()
    gpus = filtered(benches, name=workload, backend='cupy')['mean_duration'].to_numpy()
    assert len(cpus) == len(gpus)
    gpu_speedup = cpus / gpus
    df = pd.DataFrame({
        'speedup': gpu_speedup,
        'size': sizes,
    })

    fig = px.line(df, x='size', y='speedup', title=workload, log_y=True)
    fig.show()